<a href="https://colab.research.google.com/github/Ahsan97Javed/gtfs-batch-pipeline/blob/main/aggregation_gtfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTFS Batch Processing Pipeline — Aggregation Microservice

## 1. Mount Google Drive & Set Paths


In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np

drive.mount('/content/drive')

cleaned_path = '/content/drive/My Drive/GTFS_CLEANED'
agg_path = '/content/drive/My Drive/GTFS_AGGREGATED'
os.makedirs(agg_path, exist_ok=True)


Mounted at /content/drive


## 2. Load Cleaned GTFS Files


In [2]:
files = os.listdir(cleaned_path)
dfs = {}
for fname in files:
    if fname.endswith('.txt'):
        fpath = os.path.join(cleaned_path, fname)
        df = pd.read_csv(fpath, dtype=str)
        dfs[fname] = df
        print(f"Loaded {fname}, shape: {df.shape}")


Loaded calendar.txt, shape: (5553, 10)
Loaded feed_info.txt, shape: (1, 8)
Loaded stops.txt, shape: (677435, 6)
Loaded routes.txt, shape: (25081, 5)
Loaded calendar_dates.txt, shape: (13229, 3)
Loaded agency.txt, shape: (451, 5)
Loaded trips.txt, shape: (1618937, 3)
Loaded attributions.txt, shape: (3, 8)
Loaded stop_times.txt, shape: (32966240, 7)


## 3. Aggregation 1: Total Number of Trips per Route


In [3]:
# Join trips and routes to get trip count per route
trips = dfs.get('trips.txt')
routes = dfs.get('routes.txt')
if trips is not None and routes is not None:
    trip_counts = trips.groupby('route_id').size().reset_index(name='trip_count')
    trip_counts = trip_counts.merge(routes[['route_id', 'route_short_name']], on='route_id', how='left')
    trip_counts = trip_counts.sort_values(by='trip_count', ascending=False)
    trip_counts.to_csv(os.path.join(agg_path, 'trip_counts_per_route.csv'), index=False)
    print("Saved: trip_counts_per_route.csv")
    print(trip_counts.head())


Saved: trip_counts_per_route.csv
      route_id  trip_count route_short_name
9071     18162        2617               U5
21067     6386        2454               M8
23353     8443        2329              AST
15975    24376        2133              302
833      10748        2054               18


## 4. Aggregation 2: Most Popular Stops (by stop_times usage)


In [4]:
stop_times = dfs.get('stop_times.txt')
stops = dfs.get('stops.txt')
if stop_times is not None and stops is not None:
    stop_usage = stop_times.groupby('stop_id').size().reset_index(name='num_stop_times')
    stop_usage = stop_usage.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')
    stop_usage = stop_usage.sort_values(by='num_stop_times', ascending=False)
    stop_usage.to_csv(os.path.join(agg_path, 'popular_stops.csv'), index=False)
    print("Saved: popular_stops.csv")
    print(stop_usage.head())


Saved: popular_stops.csv
       stop_id  num_stop_times             stop_name
51137   172476            6624    Hamburg Bf. Altona
204417  390511            5623            Hertzallee
245861  449492            4515  F Willy-Brandt-Platz
107210  252245            3906    F Eschenheimer Tor
53371   175628            3906     F Schweizer Platz


## 5. Aggregation 3: Trips Per Day of Week (Service Availability)


In [5]:
calendar = dfs.get('calendar.txt')
if trips is not None and calendar is not None:
    trips_cal = trips.merge(calendar, on='service_id', how='left')
    days = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']
    day_counts = {day: trips_cal[trips_cal[day]=='1'].shape[0] for day in days}
    day_counts_df = pd.DataFrame(list(day_counts.items()), columns=['day_of_week', 'num_trips'])
    day_counts_df.to_csv(os.path.join(agg_path, 'trips_per_day.csv'), index=False)
    print("Saved: trips_per_day.csv")
    print(day_counts_df)


Saved: trips_per_day.csv
  day_of_week  num_trips
0      monday     982723
1     tuesday     984563
2   wednesday     969625
3    thursday     929649
4      friday     965953
5    saturday     563394
6      sunday     420810


## 6. Aggregation 4: Average Number of Stops Per Trip


In [6]:
if stop_times is not None:
    stops_per_trip = stop_times.groupby('trip_id').size()
    avg_stops_per_trip = stops_per_trip.mean()
    print(f"Average stops per trip: {avg_stops_per_trip:.2f}")
    with open(os.path.join(agg_path, 'avg_stops_per_trip.txt'), 'w') as f:
        f.write(f"Average stops per trip: {avg_stops_per_trip:.2f}")


Average stops per trip: 20.36


## 7. Any Custom Analysis: E.g., Top Agencies by Number of Routes


In [7]:
agency = dfs.get('agency.txt')
if routes is not None and agency is not None:
    agency_route_counts = routes.groupby('agency_id').size().reset_index(name='num_routes')
    agency_route_counts = agency_route_counts.merge(agency[['agency_id', 'agency_name']], on='agency_id', how='left')
    agency_route_counts = agency_route_counts.sort_values(by='num_routes', ascending=False)
    agency_route_counts.to_csv(os.path.join(agg_path, 'routes_per_agency.csv'), index=False)
    print("Saved: routes_per_agency.csv")
    print(agency_route_counts.head())


Saved: routes_per_agency.csv
    agency_id  num_routes                   agency_name
288       359        1006  Verkehrsverbund Rhein-Neckar
418         7         814     Hamburger Verkehrsverbund
105       194         804                       VGM/VRL
182       263         700     Verkehrsverbund Stuttgart
190       270         614   Verkehrsverbund Rhein-Mosel
